# Bedrock Agent - Basic Setup
Create a simple agent with action groups using boto3

In [ ]:
import boto3
import json
import time

bedrock_agent = boto3.client('bedrock-agent', region_name='us-east-1')
iam = boto3.client('iam')
lambda_client = boto3.client('lambda', region_name='us-east-1')

ACCOUNT_ID = boto3.client('sts').get_caller_identity()['Account']
AGENT_NAME = 'weather-assistant'
MODEL_ID = 'us.anthropic.claude-3-5-sonnet-20241022-v2:0'

## 1. Create IAM Role for Agent

In [ ]:
agent_role_name = f"{AGENT_NAME}-role"

trust_policy = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Principal": {"Service": "bedrock.amazonaws.com"},
        "Action": "sts:AssumeRole"
    }]
}

try:
    response = iam.create_role(
        RoleName=agent_role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy)
    )
    AGENT_ROLE_ARN = response['Role']['Arn']
    
    # Attach Bedrock model invocation policy
    iam.attach_role_policy(
        RoleName=agent_role_name,
        PolicyArn='arn:aws:iam::aws:policy/AmazonBedrockFullAccess'
    )
    
    time.sleep(10)
    print(f"✓ Created agent role: {AGENT_ROLE_ARN}")
except iam.exceptions.EntityAlreadyExistsException:
    AGENT_ROLE_ARN = iam.get_role(RoleName=agent_role_name)['Role']['Arn']
    print(f"✓ Agent role exists: {AGENT_ROLE_ARN}")

## 2. Create Lambda Function for Action Group

In [ ]:
# Create Lambda execution role
lambda_role_name = f"{AGENT_NAME}-lambda-role"
lambda_trust = {
    "Version": "2012-10-17",
    "Statement": [{"Effect": "Allow", "Principal": {"Service": "lambda.amazonaws.com"}, "Action": "sts:AssumeRole"}]
}

try:
    response = iam.create_role(RoleName=lambda_role_name, AssumeRolePolicyDocument=json.dumps(lambda_trust))
    LAMBDA_ROLE_ARN = response['Role']['Arn']
    iam.attach_role_policy(RoleName=lambda_role_name, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')
    time.sleep(10)
    print(f"✓ Created Lambda role")
except iam.exceptions.EntityAlreadyExistsException:
    LAMBDA_ROLE_ARN = iam.get_role(RoleName=lambda_role_name)['Role']['Arn']
    print(f"✓ Lambda role exists")

In [ ]:
# Lambda function code
lambda_code = b'''
import json

def lambda_handler(event, context):
    action = event['actionGroup']
    function = event['function']
    parameters = event.get('parameters', [])
    
    # Parse parameters
    params = {p['name']: p['value'] for p in parameters}
    
    if function == 'get_weather':
        city = params.get('city', 'Unknown')
        result = {"temperature": 72, "condition": "sunny", "city": city}
    elif function == 'get_forecast':
        city = params.get('city', 'Unknown')
        days = int(params.get('days', 3))
        result = {"city": city, "forecast": ["sunny", "cloudy", "rainy"][:days]}
    else:
        result = {"error": "Unknown function"}
    
    return {
        'messageVersion': '1.0',
        'response': {
            'actionGroup': action,
            'function': function,
            'functionResponse': {
                'responseBody': {
                    'TEXT': {'body': json.dumps(result)}
                }
            }
        }
    }
'''

function_name = f"{AGENT_NAME}-actions"

try:
    response = lambda_client.create_function(
        FunctionName=function_name,
        Runtime='python3.12',
        Role=LAMBDA_ROLE_ARN,
        Handler='index.lambda_handler',
        Code={'ZipFile': lambda_code},
        Timeout=60
    )
    LAMBDA_ARN = response['FunctionArn']
    print(f"✓ Created Lambda: {LAMBDA_ARN}")
except lambda_client.exceptions.ResourceConflictException:
    lambda_client.update_function_code(FunctionName=function_name, ZipFile=lambda_code)
    LAMBDA_ARN = lambda_client.get_function(FunctionName=function_name)['Configuration']['FunctionArn']
    print(f"✓ Updated Lambda: {LAMBDA_ARN}")

## 3. Create Bedrock Agent

In [ ]:
try:
    response = bedrock_agent.create_agent(
        agentName=AGENT_NAME,
        agentResourceRoleArn=AGENT_ROLE_ARN,
        foundationModel=MODEL_ID,
        instruction="You are a helpful weather assistant. Provide weather information when asked."
    )
    AGENT_ID = response['agent']['agentId']
    print(f"✓ Created agent: {AGENT_ID}")
except bedrock_agent.exceptions.ConflictException:
    agents = bedrock_agent.list_agents()['agentSummaries']
    AGENT_ID = next(a['agentId'] for a in agents if a['agentName'] == AGENT_NAME)
    print(f"✓ Agent exists: {AGENT_ID}")

## 4. Create Action Group

In [ ]:
# Define API schema
api_schema = {
    "openapi": "3.0.0",
    "info": {"title": "Weather API", "version": "1.0.0"},
    "paths": {
        "/weather": {
            "get": {
                "summary": "Get current weather",
                "operationId": "get_weather",
                "parameters": [{
                    "name": "city",
                    "in": "query",
                    "required": True,
                    "schema": {"type": "string"}
                }],
                "responses": {"200": {"description": "Success"}}
            }
        },
        "/forecast": {
            "get": {
                "summary": "Get weather forecast",
                "operationId": "get_forecast",
                "parameters": [
                    {"name": "city", "in": "query", "required": True, "schema": {"type": "string"}},
                    {"name": "days", "in": "query", "required": False, "schema": {"type": "integer"}}
                ],
                "responses": {"200": {"description": "Success"}}
            }
        }
    }
}

# Add Lambda permission
try:
    lambda_client.add_permission(
        FunctionName=function_name,
        StatementId='bedrock-agent-invoke',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:us-east-1:{ACCOUNT_ID}:agent/{AGENT_ID}"
    )
except: pass

# Create action group
response = bedrock_agent.create_agent_action_group(
    agentId=AGENT_ID,
    agentVersion='DRAFT',
    actionGroupName='weather-actions',
    actionGroupExecutor={'lambda': LAMBDA_ARN},
    apiSchema={'payload': json.dumps(api_schema)}
)

print(f"✓ Created action group: {response['agentActionGroup']['actionGroupId']}")

## 5. Prepare Agent

In [ ]:
response = bedrock_agent.prepare_agent(agentId=AGENT_ID)
print(f"✓ Agent prepared, status: {response['agentStatus']}")

# Wait for preparation
while True:
    status = bedrock_agent.get_agent(agentId=AGENT_ID)['agent']['agentStatus']
    if status == 'PREPARED':
        break
    print(f"Waiting... status: {status}")
    time.sleep(5)

print("✓ Agent ready!")

## 6. Create Agent Alias

In [ ]:
try:
    response = bedrock_agent.create_agent_alias(
        agentId=AGENT_ID,
        agentAliasName='prod'
    )
    AGENT_ALIAS_ID = response['agentAlias']['agentAliasId']
    print(f"✓ Created alias: {AGENT_ALIAS_ID}")
except:
    aliases = bedrock_agent.list_agent_aliases(agentId=AGENT_ID)['agentAliasSummaries']
    AGENT_ALIAS_ID = next(a['agentAliasId'] for a in aliases if a['agentAliasName'] == 'prod')
    print(f"✓ Alias exists: {AGENT_ALIAS_ID}")

## 7. Test Agent

In [ ]:
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name='us-east-1')

def invoke_agent(prompt):
    response = bedrock_agent_runtime.invoke_agent(
        agentId=AGENT_ID,
        agentAliasId=AGENT_ALIAS_ID,
        sessionId='test-session',
        inputText=prompt
    )
    
    result = ""
    for event in response['completion']:
        if 'chunk' in event:
            result += event['chunk']['bytes'].decode('utf-8')
    
    return result

# Test
print(invoke_agent("What's the weather in Seattle?"))

In [ ]:
print(invoke_agent("Give me a 5-day forecast for New York"))